# import lib and test GPU of tensorflow


In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import sys
import gc
import os
import tensorflow as tf
import h5py
from random import randint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold, datasets
from IPython.display import clear_output
from tsnecuda import TSNE
#import tsnecuda
#tsnecuda.test()


In [2]:
os.environ["QT_API"] = "pyqt5"
print(os.environ.get('QT_API'))


pyqt5


In [3]:
%matplotlib qt


In [5]:
#test tesnsorflow
print("tensorflow version: ",tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
# Returns whether TensorFlow was built with CUDA (GPU) support.
print("tf.test.is_Built_With_cuda = ",tf.test.is_built_with_cuda())
# RetUrns whether TensorFlow can access a GPU. (deprecated)
print("tf.test.is_gpu_available = ",tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))


tensorflow version:  2.1.0
Num GPUs Available:  1
Num CPUs Available:  1
tf.test.is_Built_With_cuda =  True
tf.test.is_gpu_available =  True


# Main program -------------->
# build data path index


In [9]:
def build_data_path_index():
    print("build data path index.....")

    ## read file path
    folder_path = '/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/VL/Mat/*'
    folder_path_list = sorted(glob.glob(folder_path))[1:] #excpet 20200507A
    # print('\n'.join(folder_name))
    file_name = []
    for path in folder_path_list: #folder_name_i = folder_name_index
        file_name += sorted(glob.glob(path+'/*'))
        # break

    file_name_A = []
    file_name_B = []
    for name in file_name:
        if name[83:84] is 'A':
            file_name_A.append(name)
        elif name[83:84] is 'B':
            file_name_B.append(name)

    print("build successful")
    return file_name_A, file_name_B

file_name_A, file_name_B = build_data_path_index()
# print('\n'.join(file_name_A))
# print('\n'.join(file_name_B))
FA_num = len(file_name_A)
FB_num = len(file_name_B)
total_num = len(file_name_A)+ len(file_name_B)
print("file_name_A number is ", len(file_name_A))
print("file_name_B number is ", len(file_name_B))
print("total data number is ", len(file_name_A)+ len(file_name_B))


build data path index.....
build successful
file_name_A number is  55
file_name_B number is  48
total data number is  103


# test data

In [7]:
num = 1
data = np.zeros((num,300,900,1000), dtype=np.float64)
for i in range(num):
    print("i=",i)
    temp = h5py.File(file_name_A[0], 'r').get('cube')
    data[i] = np.array(temp)
data = data[0][0:10]
print(type(data))
print(data.shape)


i= 0


<class 'numpy.ndarray'>
(10, 900, 1000)


In [ ]:
%matplotlib qt


In [ ]:
%matplotlib qt
def set_figure_size():
    plt.figure(figsize=(12,8))

set_figure_size()

# plt.subplot(1,3,1)
plt.axis("off")
plt.imshow(data[9], plt.cm.gray, vmax=1.0)

# plt.subplot(1,3,2)
# plt.axis("off")
# plt.imshow(data[9][0:900,0:400], plt.cm.gray, vmax=1.0)

# plt.subplot(1,3,3)
# plt.axis("off")
# plt.imshow(data[9][0:150,0:250], plt.cm.gray, vmax=1.0)


# OSP

In [ ]:
def rho(arr):
    sum=0
    for i in arr:
        sum += i**2
    r = sum**0.5
    return r
def OSPfn(M, U):
    (l, p) = M.shape
    P_U = np.identity(l) - np.dot(U,np.linalg.pinv(U));
    nu = np.zeros((p,1))
    for i in range(p):
        nu[i] = rho(P_U.dot(M[:,i]))
    return nu


In [ ]:
print(data.shape)

In [ ]:
# change data([x*y][band]) to
#        cube_arr([band][x*y])
cube_arr = data
(band, x, y) = cube_arr.shape
cube_arr = cube_arr.reshape(band, x*y)
print(cube_arr.shape)

In [ ]:
# select undesired or interfering signatures
# U =              data[:,520,900]
# U = np.stack((U, data[:,880,470]), axis=1)
U =               data[:,150,850]
U = np.vstack((U, data[:,355,850]))
# U = np.vstack((U, data[:,300,850]))
U = np.transpose(U)
print("U.shpae = ", U.shape)
print("cube_arr.shpae = ", cube_arr.shape)

threshold = 0.2
out = OSPfn(cube_arr, U).reshape((900,1000))
# out[out>=threshold] = 1
# out[out<threshold] = 0

#show result
set_figure_size()

plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(data[3], plt.cm.gray, vmax=1.0)

plt.subplot(1,2,2)
plt.axis("off")
plt.imshow(out, plt.cm.gray)


In [ ]:
# del data
%whos